In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_file
import zipfile
import os
import numpy as np
from tensorflow.keras.datasets import mnist

# 1. Baixando e Extraindo o Dataset de Gatos e Cachorros
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = get_file('cats_and_dogs_filtered.zip', origin=_URL, extract=True)
base_dir = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 2. Pré-processamento dos Dados
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)
val_data = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# 3. Criando o Modelo Inicial (do Zero)
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Saída binária (gato ou cachorro)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=10)




Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.5231 - loss: 0.7276 - val_accuracy: 0.6370 - val_loss: 0.6626
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.6078 - loss: 0.6625 - val_accuracy: 0.6380 - val_loss: 0.6375
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - accuracy: 0.6642 - loss: 0.6019 - val_accuracy: 0.6770 - val_loss: 0.5897
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.7038 - loss: 0.5615 - val_accuracy: 0.7000 - val_loss: 0.5812
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.7667 - loss: 0.4928 - val_accuracy: 0.6770 - val_loss: 0.6083
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.7992 - loss: 0.4381 - val_accuracy: 0.6960 - val_loss: 0.5902
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.8314 - loss: 0.3607 - val_accuracy: 0.7230 - val_loss: 0.5858
Epoch 8/10
63/63 ━━━━━━━

In [6]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Carregando os dados MNIST e filtrando 0 e 1
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = tf.keras.datasets.mnist.load_data()

# Filtrar apenas os dígitos 0 e 1
train_filter = (mnist_y_train == 0) | (mnist_y_train == 1)
test_filter = (mnist_y_test == 0) | (mnist_y_test == 1)

mnist_x_train, mnist_y_train = mnist_x_train[train_filter], mnist_y_train[train_filter]
mnist_x_test, mnist_y_test = mnist_x_test[test_filter], mnist_y_test[test_filter]

# Normalizando e redimensionando para a entrada da rede
mnist_x_train = np.expand_dims(mnist_x_train, axis=-1) / 255.0
mnist_x_test = np.expand_dims(mnist_x_test, axis=-1) / 255.0

# Convertendo para RGB
mnist_x_train = tf.image.grayscale_to_rgb(tf.image.resize(mnist_x_train, [128, 128]))
mnist_x_test = tf.image.grayscale_to_rgb(tf.image.resize(mnist_x_test, [128, 128]))

# Adaptando o modelo pré-treinado (exemplo com VGG16)
base_model = VGG16(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Congelar o modelo base

# Adicionando camadas ao modelo
model = Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Saída binária
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando no MNIST
model.fit(mnist_x_train, mnist_y_train, validation_data=(mnist_x_test, mnist_y_test), epochs=5)

# Avaliação no MNIST
loss, accuracy = model.evaluate(mnist_x_test, mnist_y_test)
print(f"Acurácia no MNIST (0 e 1): {accuracy * 100:.2f}%")


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
396/396 ━━━━━━━━━━━━━━━━━━━━ 42s 84ms/step - accuracy: 0.9890 - loss: 0.0211 - val_accuracy: 1.0000 - val_loss: 3.3650e-05
Epoch 2/5
396/396 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - accuracy: 1.0000 - loss: 2.5298e-05 - val_accuracy: 1.0000 - val_loss: 1.8468e-05
Epoch 3/5
396/396 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 1.0000 - loss: 7.2696e-06 - val_accuracy: 1.0000 - val_loss: 1.0896e-05
Epoch 4/5
396/396 ━━━━━━━━━━━━━━━━━━━━ 19s 48ms/step - accuracy: 1.0000 - loss: 5.4632e-06 - val_accuracy: 1.0000 - val_loss: 8.9693e-06
Epoch 5/5
396/396 ━━━━━━━━━━━━━━━━━━━━ 19s 47ms/step - accuracy: 1.0000 - loss: 2.9128e-06 - val_accuracy: 1.0000 - val_loss: 2.2789e-05
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 1.0000 - loss: 1.1780e-05
Acurácia no MNIST (0 e 1): 100.00%
